### Dynamic fetchAPI to chose how many files

In [1]:
import boto3
import json
import requests
import csv
from io import StringIO

s3_client = boto3.client("s3")
LOCAL_FILE_SYS = "/tmp"
S3_EXTERNAL_BUCKET = "2202934b-external"
S3_LAKE_BUCKET = "openfisheries-lake"
API_LINK = "https://www.openfisheries.org/api/landings/"

def fetch_from_s3(s3_bucket, s3_key, target_column_index=4):
    try:
        response = s3_client.get_object(Bucket=s3_bucket, Key=s3_key)
        content = response['Body'].read().decode('utf-8')
        
        a3_codes = []
        csv_reader = csv.reader(StringIO(content))
        for row in csv_reader:
            if len(row) > target_column_index:
                a3_code = row[target_column_index].strip()
                a3_codes.append(a3_code)
        
        return a3_codes

    except:
        return None

def fetch_endpoint(a3_code):
    try:
        response = requests.get(f"{API_LINK}species/{a3_code}.json")
        response.raise_for_status()  # Raise an HTTPError for bad responses
        data = response.json()
        return data

    except requests.RequestException as e:
        print(f"Error fetching endpoint for {a3_code}: {str(e)}")
        return None

def lambda_handler(event, context):
    a3_codes = fetch_from_s3(S3_EXTERNAL_BUCKET, 'species_in_USA.csv')
    LIMIT_ROWS = event.get("LIMIT_ROWS", 30)
    
    try:
        response = requests.get(f"{API_LINK}countries/USA.json")
        response.raise_for_status()  # Raise an HTTPError for bad responses
        data = response.json()
        s3_client.put_object(Body=json.dumps(data), Bucket=S3_LAKE_BUCKET, Key='USA.json')
        
    except requests.RequestException as e:
        print(f"Error fetching endpoint for USA: {str(e)}")
        return None
    
    
    if a3_codes is not None:
        for a3_code in a3_codes:
            api_data = fetch_endpoint(a3_code)
            
            if api_data is not None:
                s3_key = f"species/{a3_code}.json"
                s3_client.put_object(Body=json.dumps(api_data), Bucket=S3_LAKE_BUCKET, Key=s3_key)
                print(f"File {s3_key} successfully uploaded to {S3_LAKE_BUCKET}")
                LIMIT_ROWS -= 1
            if LIMIT_ROWS < 1:
                return {
                    'statusCode': 200,
                    'body': json.dumps('Required species retrieved!')
                }

        return {
            'statusCode': 200,
            'body': json.dumps('All species retrieved!')
        }
    
    else:
        raise ValueError("Failed to fetch CSV data from S3")

### Wrangler

In [2]:
import boto3
import pandas as pd
from io import StringIO
import json
import logging

# Configure logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

def clean_data(df):
    try:
        df.dropna(inplace=True)
        df.drop_duplicates(inplace=True)
        for column in df.columns:
            if df[column].dtype == 'object':
                df[column] = df[column].str.strip().apply(lambda x: ' '.join(word.capitalize() for word in x.split()))
                df[column] = df[column].apply(lambda x: x.upper() if len(x) <= 3 else x)
    except Exception as e:
        logger.error(f"Error cleaning data: {str(e)}")

    return df

def process_s3_data(input_bucket, input_folder, output_column):
    s3 = boto3.client('s3')
    df = pd.DataFrame()

    try:
        response = s3.list_objects(Bucket=input_bucket, Prefix=f'{input_folder}/')

        for obj in response.get('Contents', []):
            file_key = obj['Key']
            KEY = file_key.split('/')[-1].replace('.json', '')
            response = s3.get_object(Bucket=input_bucket, Key=file_key)
            json_data = response['Body'].read().decode('utf-8')
            temp_df = pd.read_json(StringIO(json_data))
            temp_df[output_column] = KEY
            df = pd.concat([df, temp_df], ignore_index=True)

    except Exception as e:
        logger.error(f"Error processing S3 data: {str(e)}")

    return df

def to_warehouse(df, folder_name, output_bucket):
    try:
        s3 = boto3.client('s3')
        csv_data = df.to_csv(index=False)
        csv_key = f'{folder_name}.csv'
        s3.put_object(Bucket=output_bucket, Key=f"{csv_key}/{csv_key}", Body=csv_data)
        logger.info(f'CSV file created successfully in bucket: {output_bucket}')
    except Exception as e:
        logger.error(f"Error uploading CSV to warehouse: {str(e)}")
        

def process_usa_data(species_df, lake_bucket, usa_file):
    try:
        s3 = boto3.client('s3')

        usa_obj = s3.get_object(Bucket=lake_bucket, Key=usa_file)
        usa_data = usa_obj['Body'].read().decode('utf-8')

        usa_df = pd.read_json(StringIO(usa_data))
        usa_yearly = usa_df.groupby('year')['catch'].sum().reset_index()
        total_catch_species = species_df.groupby('year')['catch'].sum().reset_index()
        
        result_df = pd.merge(usa_yearly, total_catch_species, on='year', how='left', suffixes=('_usa', '_species'))
        result_df['catch'] = result_df['catch_usa'] - result_df['catch_species'] # catch of other species besides the species in species list

        other_species_df = pd.DataFrame({'a3_code': ['OTHERS'] * len(result_df), 'catch': result_df['catch'], 'year': result_df['year']})
        species_df = pd.concat([species_df, other_species_df], ignore_index=True)

        return species_df

    except Exception as e:
        logger.error(f"Error processing USA data: {str(e)}")
        return pd.DataFrame()


def lambda_handler(event, context):
    LOOKUP = '2202934b-external'
    LAKE = 'openfisheries-lake'
    WAREHOUSE = 'openfisheries-warehouse'

    s3_client = boto3.client('s3')
    response = s3_client.get_object(Bucket=LOOKUP, Key='species_in_USA.csv')
    csv_data = response['Body'].read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_data))
    df = clean_data(df)
    df = pd.concat([df, pd.DataFrame({'scientific_name': ['OTHERS'],
                                       'state': [' '],
                                       'english_name': ['OTHERS'],
                                       'taxocode': [' '],
                                       'a3_code': ['OTHERS'],
                                       'isscaap': [' ']})], ignore_index=True)

    to_warehouse(df, 'dim_species', WAREHOUSE)

    try:
        species_df = process_s3_data(LAKE, 'species', 'a3_code')
        species_df = clean_data(species_df)
        usa_species_df = process_usa_data(species_df, LAKE, 'USA.json')
        to_warehouse(usa_species_df, 'fact_species', WAREHOUSE)

        return {
            'statusCode': 200,
            'body': json.dumps('Processing completed successfully!')
        }
    except Exception as e:
        logger.error(f"Error in lambda_handler: {str(e)}")
        return {
            'statusCode': 500,
            'body': json.dumps('Error during processing. Check logs for details.')
        }

### SQL EDA

In [3]:
CREATE VIEW species AS (
SELECT d.state, d.scientific_name, d.english_name, d.isscaap, d.taxocode, f.*
FROM dim_species_csv d, fact_species_csv f
WHERE d.a3_code = f.a3_code
)

SyntaxError: invalid syntax (233434155.py, line 1)